In [10]:
# 多输入多输出通道
import torch
from d2l import torch as d2l

实践中，RGB 图像是 $3 * h * w$ 的三维张量,值为 $3$ 的那一个维度称为**通道**。

## 1. 多输入通道
输入数据有 $c_i$ 个通道，就该有 $c_i$ 个卷积核。
第 $i$ 个通道上的对应 input 和 kernel 做卷积运算,得到结果矩阵 $Y_i$。
单输出：$Z =  \sum Y_i + b$
> 输入数据有 $c_i$ 个通道，$K$ 的维度就应该为 $c_i * k_h * k_w$. 表示 $c_i$ 个高和宽为 $k_h$ 和 $k_w$ 的 kernel 。

In [11]:
# 对每个通道执行互相关操作，然后将结果相加。
def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [12]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

X.shape,K.shape,corr2d_multi_in(X, K)

(torch.Size([2, 3, 3]),
 torch.Size([2, 2, 2]),
 tensor([[ 56.,  72.],
         [104., 120.]]))

## 2. 多输出通道 $c_o$
在上一小节，多个通道的卷积输出用`sum()`合成为一个单输出。
在最流行的神经网络架构中，随着神经网络层数的加深，我们常会增加输出通道的维数。
对每个输出通道，都应该有一个 $c_i * k_h * k_w$ 的卷积核。
对于有 $c_o$ 个输出通道的卷积网络，卷积核尺寸应当为 $c_o * c_i * k_h * k_w$

In [13]:
# 生成 3 输出通道的卷积核
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [14]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [15]:
corr2d_multi_in_out(X, K),corr2d_multi_in_out(X, K).shape

(tensor([[[ 56.,  72.],
          [104., 120.]],
 
         [[ 76., 100.],
          [148., 172.]],
 
         [[ 96., 128.],
          [192., 224.]]]),
 torch.Size([3, 2, 2]))

## 3. $1*1$ 卷积核
$1*1$ 卷积核的唯一作用是控制输出的**通道数**。
我们可以将 $1*1$ 卷积层看作在每个像素位置应用的全连接层。
举例：
如果输入数据有 3 个通道，同时希望输出数据有 2 个通道，可以设置 $1*1$ 卷积核为：
(2,3,1,1)

In [16]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [17]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

# 两者是等价的
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6